In [48]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [56]:
with open('013_b_1114_1_2_p1.csv', newline='') as csvfile1:
    data1 = pd.read_csv(csvfile1)
    data1 = data1.rename(columns = {'p1Cash':'Cash','p1Stock':'Stock','p1TotalAsset':'TotalAsset',
                                   'p1Decision':'Decision','p1ChechHistory':'CheckHistory'})
with open('014_b_1114_1_2_p2.csv', newline='') as csvfile2:
    data2 = pd.read_csv(csvfile2)
    data2 = data2.rename(columns = {'p2Cash':'Cash','p2Stock':'Stock','p2TotalAsset':'TotalAsset',
                                   'p2Decision':'Decision','p2ChechHistory':'CheckHistory'})

In [57]:
X = data1.loc[:,['Cash','StockPrice','TotalAsset']]
X['opp_TotalAsset'] = data2.loc[:,['TotalAsset']]
X['CheckHistory'] = pd.get_dummies(data1.loc[:,'CheckHistory'])['yes']

In [58]:
X_1_101 = X[1:101]
X_1_101 = X_1_101.reset_index()
X_0_100 = X[0:100]
X_0_100 = X_0_100.rename(columns = {'Cash':'pre_Cash','StockPrice':'pre_StockPrice','TotalAsset':'pre_TotalAsset',
                                   'opp_TotalAsset':'pre_opp_TotalAsset','CheckHistory':'pre_CheckHistory'}, inplace = False)
X = pd.concat([X_1_101,X_0_100], axis=1)
X = X.drop(['index'], axis = 1)

In [59]:
X['delta_Price'] = X['StockPrice']-X['pre_StockPrice']
X_data = X[0:99]
X_norm = X_data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

In [60]:
Y_data = data1.loc[:,'Decision']
Y_dum = pd.get_dummies(Y_data)
Y = Y_dum[1:100]

In [61]:
def buildTrain(X_norm, Y, past = 30, future = 5):
    X_train, Y_train = [], []
    for i in range(X_norm.shape[0]-future-past):
        X_train.append(np.array(X_norm.iloc[i:i+past]))
    for i in range(Y.shape[0]-future):
        Y_train.append(np.array(Y.iloc[i:i+future]))
    return np.array(X_train), np.array(Y_train)

In [45]:
# def shuffle(X,Y):
#     np.random.seed(408)
#     randomList = np.arange(X.shape[0])
#     np.random.shuffle(randomList)
#     return X[randomList], Y[randomList]

In [68]:
X_train, Y_train = buildTrain(X_norm, Y, 30, 1)

# X_train, Y_train = shuffle(X_train, Y_train)

X_train, X_val = train_test_split(X_train, test_size = 0.1, random_state = 408)
Y_train, Y_val = train_test_split(Y_train, test_size = 0.1, random_state = 408)
print(Y_train)

[[[0 1 0]]

 [[1 0 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[1 0 0]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[1 0 0]]

 [[1 0 0]]

 [[0 0 1]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[1 0 0]]

 [[0 0 1]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 [[1 0 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 0 1]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 1 0]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 0 1]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[1 0 0]]

 [[0 1 0]]

 [[1 0 0]]

 [[1 0 0]]

 [[0 1 0]]

 [[0 0 1]]

 [[0 0 1]]

 [[0 0 1]]

 [[1

In [64]:
def buildModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length = shape[1], input_dim = shape[2]))
    model.add(Dense(3))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model


In [67]:
X_train, Y_train = buildTrain(X_norm, Y, 30, 1)

X_train, Y_train = shuffle(X_train, Y_train)

X_train, X_val = train_test_split(X_train, test_size = 0.1, random_state = 408)
Y_train, Y_val = train_test_split(Y_train, test_size = 0.1, random_state = 408)

model = buildModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                880       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 33        
Total params: 913
Trainable params: 913
Non-trainable params: 0
_________________________________________________________________


D:\Software\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Software\anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 11))`
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: Error when checking target: expected dense_3 to have 2 dimensions, but got array with shape (61, 1, 3)